In [3]:
import pandas as pd
# 시각화에 필요한 라이브러리 임포트
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# 내부에 결과를 출력하도록 설정
%matplotlib inline

In [4]:
train = pd.read_csv('/content/drive/MyDrive/bus_rider/train.csv')
test = pd.read_csv('/content/drive/MyDrive/bus_rider/test.csv')
bts = pd.read_csv('/content/drive/MyDrive/bus_rider/bus_bts.csv')

In [5]:
# 학습 데이터와 테스트 데이터를 구분하기 위한 변수 생성
train['cue'] = 0
test['cue'] = 1

# 학습 데이터와 테스트 데이터 통합
df = pd.concat([train, test], axis=0)

In [6]:
# datetime 변수형으로 변환
df['date'] = pd.to_datetime(df['date'])
# 요일 추출 (0-월요일~6-일요일)
df['weekday'] = df['date'].dt.weekday

In [7]:
df[['weekday']].head()

,weekday
0,6
1,6
2,6
3,6
4,6


In [8]:
# 요일별 평균 탑승 승객수를 구하는 함수
def week_mean() :
    # 전체 데이터에서 train 데이터에 해당하는 행 추출
    train_data = df.query('cue==0').reset_index(drop=True)
    
    # 일괄적으로 1의 값을 가지는 'weekdaymean'변수 생성
    df['weekdaymean'] = 1

    # 각 요일에 해당하는 인덱스 추출
    index0 = df.query('weekday==0').index
    index1 = df.query('weekday==1').index
    index2 = df.query('weekday==2').index
    index3 = df.query('weekday==3').index
    index4 = df.query('weekday==4').index
    index5 = df.query('weekday==5').index
    index6 = df.query('weekday==6').index

    # 인덱스를 활용하여 'weekdaymean'의 값을 각 요일에 맞는 평균 탑승 승객수로 변경 
    df.iloc[index0,-1] = train_data.query('weekday==0')['18~20_ride'].mean()
    df.iloc[index1,-1] = train_data.query('weekday==1')['18~20_ride'].mean()
    df.iloc[index2,-1] = train_data.query('weekday==2')['18~20_ride'].mean()
    df.iloc[index3,-1] = train_data.query('weekday==3')['18~20_ride'].mean()
    df.iloc[index4,-1] = train_data.query('weekday==4')['18~20_ride'].mean()
    df.iloc[index5,-1] = train_data.query('weekday==5')['18~20_ride'].mean()
    df.iloc[index6,-1] = train_data.query('weekday==6')['18~20_ride'].mean()
    
    return df

# 함수를 실행하여 변수 생성
df = week_mean()

In [9]:
df[['weekdaymean']].head()

,weekdaymean
0,1.034282
1,1.034282
2,1.034282
3,1.034282
4,1.034282


In [10]:
# 버스 종류별 평균 탑승 승객수를 구하는 함수
def inout_mean() :
    # 전체 데이터에서 train 데이터에 해당하는 행 추출
    train_data = df.query('cue==0').reset_index(drop=True)
    
    # 일괄적으로 1의 값을 가지는 in_out_mean 변수 생성
    df['in_out_mean'] = 1

    # 버스 종류별 인덱스 추출
    in_index = df.query('in_out == "시내"').index # 시내버스
    out_index = df.query('in_out == "시외"').index # 시외버스

    # 인덱스를 활용하여 in_out_mean의 값을 각 버스 종류에 맞는 
    # 평균 탑승 승객수로 변경
    df.iloc[in_index,-1] = train_data.query('in_out == "시내"')['18~20_ride'].mean()
    df.iloc[out_index,-1] = train_data.query('in_out == "시외"')['18~20_ride'].mean()
    
    return df

# 함수를 실행하여 변수 생성
df = inout_mean()

In [11]:
df[['in_out_mean']].head()

,in_out_mean
0,2.044345
1,2.044345
2,2.044345
3,1.228499
4,1.228499


In [12]:
# 날짜별 오전시간에 탑승한 총 승객수
f=df.groupby('date')['6~7_ride','7~8_ride','8~9_ride','9~10_ride','10~11_ride'
                    ].sum().reset_index()

# 변수명 바꾸기
f.columns=['date','6~7_ride_sum','7~8_ride_sum','8~9_ride_sum','9~10_ride_sum',
           '10~11_ride_sum']

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, f, how='left', on='date')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [13]:
df[['date','6~7_ride_sum','7~8_ride_sum','8~9_ride_sum','9~10_ride_sum',
    '10~11_ride_sum']].head()

,date,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum
0,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
1,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
2,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
3,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
4,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0


In [14]:
#bts 테이블을 통한 변수
# 배차간격
# 탑승 날짜를 datetime형으로 변환
bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

# 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호 기준으로 정렬
f = bts.sort_values(by=['geton_date','geton_station_code','bus_route_id',
                        'geton_time2'],ascending=True).reset_index()

In [15]:
# 반복문 실행 시간이 많이 소요되어 상태 진행률을 확인할 수 있는 tqdm 라이브러리 사용
from tqdm import notebook

# 시차를 넣기 위한 빈 리스트 생성
interval_lst = []
for i in notebook.tqdm(range(0,len(f)-1)):
    # 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호가 같다면
    if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) & 
        (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) & 
        (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):
        # 탑승 날짜의 차를 interval_lst에 넣기
        interval_lst.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)
    else:
       # 같지 않다면 0을 interval_lst에 넣기
        interval_lst.append(0)

interval_lst.insert(0, '0')

  0%|          | 0/173852 [00:00<?, ?it/s]

In [16]:
f['interval'] = interval_lst # interval_lst를 'interval'변수로 생성 
# 'interval' 변수의 값을 문자형으로 변환 후 '시:분:초'의 형태로 변환
f['interval'] = f['interval'].astype(str).str[7:].replace('','00:00:00')
f['bus_route_id'] = f['bus_route_id'].astype(object)

In [17]:
# 분으로 변환하는 함수
def get_min(time_str):
    h, m, s = time_str.split(':')
    return (int(h) * 3600 + int(m) * 60 + int(s)) / 60
# 'interval' 함수의 모든 값에 대해 함수 적용하여 나온 값을 time리스트에 넣기
time = []
for i in f['interval'] :
    time.append(get_min(i))

f['time'] = time # time리스트를 'time' 변수로 생성
f['time'] = f['time'].astype(int) # 'time' 변수를 int 타입으로 변경
f = f[f['time'] > 3] # 간격이 3분보다 작은 것 제외 
f = f[f['time'] < 180] # 간격이 3시간보다 큰 것 제외

In [18]:
f.shape

(41982, 17)

In [19]:
# 버스 노선별 버스 간격의 평균 'bus_interval’변수 생성
data = f.groupby('bus_route_id')['time'].agg([('bus_interval', 'mean')]
                                            ).reset_index()
data['bus_interval'] = data['bus_interval'].astype(int)

# 변수 생성에 많은 시간이 소요되므로 csv 파일로 저장
data.to_csv('bus_interval.csv', index = False)

In [20]:
# csv 파일이 성공적으로 저장됐다면 다음 과정부터 진행
import numpy as np
df['bus_route_id'] = df['bus_route_id'].astype(np.int64)
bus_interval = pd.read_csv("bus_interval.csv")

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, bus_interval, how = 'left', on = 'bus_route_id')
df['bus_interval'] = df['bus_interval'].fillna(9999) # 병합 중 생기는 NA값 처리

In [21]:
df[['bus_interval']].head()

,bus_interval
0,47.0
1,47.0
2,47.0
3,47.0
4,47.0


In [25]:
#수요가 많을 것으로 예상되는 정류장
# 학교, 제주도..(호텔.. 공항..)
# 정류장 이름만 unique하게 출력
pd.DataFrame( np.unique( df['station_name']) )

,0
0,(구)구판장
1,(구)중문동주민센터
2,(구)중앙파출소
3,(구)화산초등학교
4,1100고지휴게소
...,...
1966,흙통
1967,흥국사
1968,흥덕사
1969,희진주유소


In [57]:
hightschool = []
for stname in df['station_name']:
  if('고등학교' in stname):
    hightschool.append(stname)

hightschool[:6]

['오현중고등학교',
 '신흥리(마을입구)함덕고등학교',
 '오현중고등학교',
 '신흥리(마을입구)함덕고등학교',
 '한림고등학교',
 '애월고등학교']

In [87]:
hightschool = [stname for stname in df['station_name'] if '고등학교' in stname]
unviersity = [stname for stname in df['station_name'] if '대학교' in stname]
# 각 리스트에서 중복을 제거
hightschool = np.unique(hightschool)
unviersity = np.unique(unviersity)

In [77]:
# 고등학교 가 포함된 row를 찾는다.
def findHightSchool(x):
  if '고등학교' in x:
    return x

def finduniversity(x):
  if '대학교' in x:
    return x
#버스정류장이 고등학교가 포함된 row
highschool_index = df['station_name'].apply(findHightSchool )
#버스정류장이 대학교가 포함된 row
unversity_index = df['station_name'].apply(finduniversity )

In [81]:
hightschool_sation =  df.iloc[highschool_index[highschool_index.notnull()].index, :]
unversity_station =  df.iloc[unversity_index[unversity_index.notnull()].index, :]

In [82]:
hightschool_sation.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,cue,weekday,weekdaymean,in_out_mean,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum,bus_interval
100,100,2019-09-01,21000000,시내,424,오현중고등학교,33.51561,126.55861,0.0,0.0,...,0,6,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,61.0
119,119,2019-09-01,21010000,시내,402,신흥리(마을입구)함덕고등학교,33.54360,126.65364,0.0,0.0,...,0,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,51.0
121,121,2019-09-01,21010000,시내,423,오현중고등학교,33.51518,126.55707,0.0,0.0,...,0,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,51.0
150,150,2019-09-01,21070000,시내,402,신흥리(마을입구)함덕고등학교,33.54360,126.65364,0.0,3.0,...,0,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,37.0
159,159,2019-09-01,21080000,시내,737,한림고등학교,33.39918,126.25841,0.0,0.0,...,0,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,88.0


In [83]:
unversity_station.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,cue,weekday,weekdaymean,in_out_mean,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum,bus_interval
236,236,2019-09-01,21220000,시내,121,제주대학교병원,33.46909,126.54739,0.0,0.0,...,0,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,76.0
238,238,2019-09-01,21220000,시내,207,제주대학교입구,33.45672,126.55076,0.0,0.0,...,0,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,76.0
252,252,2019-09-01,21230000,시내,696,제주대학교입구,33.45673,126.55105,0.0,0.0,...,0,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,88.0
326,326,2019-09-01,21400000,시내,120,제주대학교병원,33.46916,126.54777,0.0,0.0,...,0,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,118.0
328,328,2019-09-01,21400000,시내,696,제주대학교입구,33.45673,126.55105,0.0,0.0,...,0,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,118.0


In [89]:
def schoolstation(x):
  if x in hightschool:
    return 1
  elif x in unviersity:
    return 1
  else:
    return 0

df['school'] =  df['station_name'].apply(schoolstation)
  

In [90]:
df.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,weekday,weekdaymean,in_out_mean,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum,bus_interval,school
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,6,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,6,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,6,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,6,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0


In [99]:
# 환승 공항 터미널..
# [sn for sn in df['station_name'] if '환승' in sn]
temp_index = df['station_name'].apply(lambda x : '환승' in x)
transfer_station  = df[temp_index.values]

temp_index = df['station_name'].apply(lambda x : '공항' in x)
airport_station  = df[temp_index.values]

temp_index = df['station_name'].apply(lambda x : '터미널' in x)
terminal_station  = df[temp_index.values]


In [102]:
transfer =  list(transfer_station['station_name'].unique())
airport =  list(airport_station['station_name'].unique())
terminal =  list(terminal_station['station_name'].unique())

In [103]:
def transferstation(x):
  if x in transfer:
    return 1
  elif x in airport:
    return 1
  elif x in terminal:
    return 1
  else:
    return 0

df['transfer'] =  df['station_name'].apply(transferstation)

In [104]:
df.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,weekdaymean,in_out_mean,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum,bus_interval,school,transfer
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0,0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0,0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0,0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0,1
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,1.034282,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0,0


In [124]:
from datetime import date
# 날짜에서 연휴 구분
# df[df['date'].astype(str) == '2019-09-12'] # 데이터 확인

def findhollyday(x):
  if x in ['2019-09-12','2019-09-14','2019-10-03','2019-10-09']:
    return 1
  else: 
    return 0

booleanindex = df['date'].astype(str).apply(findhollyday)
df['hollyday'] = booleanindex
df.head()


,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,in_out_mean,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum,bus_interval,school,transfer,hollyday
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0,0,0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0,0,0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0,0,0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0,1,0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,1.228499,2942.0,4518.0,5682.0,6275.0,6774.0,47.0,0,0,0


In [126]:
df.columns

Index(['id', 'date', 'bus_route_id', 'in_out', 'station_code', 'station_name',
       'latitude', 'longitude', '6~7_ride', '7~8_ride', '8~9_ride',
       '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff',
       '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
       '18~20_ride', 'cue', 'weekday', 'weekdaymean', 'in_out_mean',
       '6~7_ride_sum', '7~8_ride_sum', '8~9_ride_sum', '9~10_ride_sum',
       '10~11_ride_sum', 'bus_interval', 'school', 'transfer', 'hollyday'],
      dtype='object')

In [133]:
# 오전시간대의 승차한 승객수의 합
df['ride_sum'] =  df.loc[:,['6~7_ride', '7~8_ride', '8~9_ride','9~10_ride', '10~11_ride', '11~12_ride']].sum(axis=1)
# 오전시간대의 하차한 승객수의 합
df['takeoff_sum'] =  df.loc[:,['6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff']].sum(axis=1)

In [134]:
df.loc[:,['ride_sum','takeoff_sum']].head()

,ride_sum,takeoff_sum
0,16.0,0.0
1,22.0,0.0
2,4.0,0.0
3,79.0,0.0
4,0.0,1.0


In [137]:
#  df[['bus_route_id','station_code']]
df['route_station'] = df['bus_route_id'].astype(str) + ','+df['station_code'].astype(str)

In [140]:
# 오전 오후를 2시간 간격으로 인원수의 합
df['68a'] =  df['6~7_ride'] + df['7~8_ride']
df['810a'] =  df['8~9_ride'] + df['9~10_ride']
df['1012a'] =  df['10~11_ride'] + df['11~12_ride']

df['68b'] =  df['6~7_takeoff'] + df['7~8_takeoff']
df['810b'] =  df['8~9_takeoff'] + df['9~10_takeoff']
df['1012b'] =  df['10~11_takeoff'] + df['11~12_takeoff']

In [141]:
# 버스노선별정류장별 오전시간의 총 승객수와 평균 10~12
data = df.groupby(['route_station'])['1012a'].agg({'sum','mean'}).reset_index()

In [143]:
# 컬럼명 변경(부분 변경)
data.rename(columns={'mean' : '1021a_mean', 'sum':'1012a_sum'} ,inplace=True)

In [146]:
df = pd.merge(df,data, how='left',on='route_station')

In [147]:
df.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,takeoff_sum,route_station,68a,810a,1012a,68b,810b,1012b,1021a_mean,1012a_sum
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,0.0,"4270000,344",1.0,7.0,8.0,0.0,0.0,0.0,5.304348,244.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,0.0,"4270000,357",5.0,6.0,11.0,0.0,0.0,0.0,11.608696,534.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,0.0,"4270000,432",2.0,2.0,0.0,0.0,0.0,0.0,2.304348,106.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,0.0,"4270000,1579",17.0,32.0,30.0,0.0,0.0,0.0,36.673913,1687.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,1.0,"4270000,1646",0.0,0.0,0.0,0.0,1.0,0.0,0.564103,22.0


In [162]:
#카테고리별 승객의 합
f = bts.groupby( ['bus_route_id','user_category'])['user_count'].agg([('승객수','sum')]).reset_index()
g = pd.pivot_table(f, values='승객수', index = 'bus_route_id', columns='user_category', fill_value=0).reset_index()
g.head()

user_category,bus_route_id,1.0,2.0,4.0,6.0,27.0,28.0,29.0,30.0
0,4270000,443,3,2,0,0,0,0,0
1,4280000,664,5,2,0,0,0,0,0
2,8170000,314,0,4,0,0,0,0,0
3,8180000,352,1,0,0,0,0,0,0
4,17010000,6,0,0,0,0,0,0,0


In [164]:
# 컬럼명 변겅(전체)
g.columns = ['bus_route_id', 'adult', 'kids','teen', 'elder', 'jang', 'jang2', 'ugong', 'ugong2']

In [165]:
g = g[['bus_route_id', 'adult', 'kids','teen', 'elder']]
g.head()

,bus_route_id,adult,kids,teen,elder
0,4270000,443,3,2,0
1,4280000,664,5,2,0
2,8170000,314,0,4,0
3,8180000,352,1,0,0
4,17010000,6,0,0,0


In [166]:
# 기존 데이터 프레임에 새로운 변수를 통합
df = pd.merge(df,g, how='left', on = 'bus_route_id')
df.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,1012a,68b,810b,1012b,1021a_mean,1012a_sum,adult,kids,teen,elder
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,8.0,0.0,0.0,0.0,5.304348,244.0,443.0,3.0,2.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,11.0,0.0,0.0,0.0,11.608696,534.0,443.0,3.0,2.0,0.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,0.0,0.0,0.0,0.0,2.304348,106.0,443.0,3.0,2.0,0.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,30.0,0.0,0.0,0.0,36.673913,1687.0,443.0,3.0,2.0,0.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,1.0,0.0,0.564103,22.0,443.0,3.0,2.0,0.0
